"msmarco-passage"

A passage ranking benchmark with a collection of 8.8 million passages and question queries. Most relevance judgments are shallow (typically at most 1-2 per query), but the TREC Deep Learning track adds deep judgments. Evaluation typically conducted using MRR@10.

In [69]:
import ir_datasets
import logging
import random
import re
import spacy
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Carrega o modelo de NLP para reconhecimento de entidades
nlp = spacy.load("en_core_web_sm")

NUM_QUERIES_PER_TYPE = 200  # Ajuste conforme necessário
SIMILARITY_THRESHOLD = 0.7  # Limite para considerar queries como muito parecidas

query_types = {"PERSON": [], "LOCATION": [], "DESCRIPTION": [], "NUMERIC": [], "ENTITY": []}
queries_annotated = []  # Lista de queries com anotações
selected_queries = []   # Queries selecionadas

# Define o nome do dataset
dataset_name = "msmarco-passage-v2/train"

try:
    dataset = ir_datasets.load(dataset_name)
    print("Dataset carregado com sucesso!")
except Exception as e:
    print(f"Erro ao carregar o dataset {dataset_name}: {e}")
    dataset = None

def decode_utf8_safe(text):
    """Tenta converter texto para UTF-8 de forma segura, ignorando caracteres não válidos."""
    try:
        return text.encode("utf-8").decode("utf-8")
    except UnicodeDecodeError:
        return text.encode("utf-8", "ignore").decode("utf-8", "ignore")

def classify_query(text):
    """Classifica a query automaticamente em uma das categorias: NUMERIC, ENTITY, LOCATION, PERSON, DESCRIPTION."""
    
    # Regras baseadas em palavras-chave
    if re.search(r'\b(who|whom)\b', text, re.IGNORECASE):
        return "PERSON"
    if re.search(r'\b(where|location|city|country)\b', text, re.IGNORECASE):
        return "LOCATION"
    if re.search(r'\b(what|define|describe|meaning)\b', text, re.IGNORECASE):
        return "DESCRIPTION"
    if re.search(r'\b(how many|how much|\d+|percent|percentage)\b', text, re.IGNORECASE):
        return "NUMERIC"

    # Usa NLP para análise de entidades
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in ["PERSON"]:
            return "PERSON"
        elif ent.label_ in ["GPE", "LOC"]:  # GPE = Países, cidades, estados | LOC = Locais
            return "LOCATION"
        elif ent.label_ in ["CARDINAL", "QUANTITY", "PERCENT", "MONEY"]:
            return "NUMERIC"
        elif ent.label_ in ["ORG", "PRODUCT", "EVENT"]:
            return "ENTITY"

    # Se não se encaixar em nenhuma categoria específica, assume DESCRIPTION
    return "DESCRIPTION"

# Processando queries
if dataset:
    try:
        for query in dataset.queries_iter():
            text_utf8 = decode_utf8_safe(query.text)

            if text_utf8 == query.text:  # Apenas queries válidas
                q_type = classify_query(text_utf8)  # Classificação automática
                queries_annotated.append((query.query_id, text_utf8, q_type))
                query_types[q_type].append((query.query_id, text_utf8))
            else:
                logging.warning(f"Query ID {query.query_id} contém caracteres inválidos e foi ignorada.")
    except Exception as e:
        logging.error(f"Erro ao iterar sobre as queries: {e}")

# Selecionando queries proporcionalmente
for q_type, queries in query_types.items():
    # Filtra apenas as queries anotadas do tipo correspondente
    filtered_queries = [q for q in queries_annotated if q[2] == q_type]

    # Seleciona aleatoriamente queries desse tipo, garantindo que cada uma contenha (id, texto, tipo)
    selected_queries.extend(random.sample(filtered_queries, min(NUM_QUERIES_PER_TYPE, len(filtered_queries))))


# Criando matriz TF-IDF para medir similaridade
query_texts = [query_text for _, query_text,_ in selected_queries]
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(query_texts)

# Calculando similaridade entre todas as queries selecionadas
similarity_matrix = cosine_similarity(tfidf_matrix)

# Identificando queries muito similares
to_replace = set()
for i in range(len(selected_queries)):
    for j in range(i + 1, len(selected_queries)):
        if similarity_matrix[i, j] > SIMILARITY_THRESHOLD:
            print(f"🔄 Queries muito similares detectadas:")
            print(f"  [1] {selected_queries[i][1]}")
            print(f"  [2] {selected_queries[j][1]}")
            print("-" * 60)
            to_replace.add(j)  # Marca a segunda query como redundante

# Substituindo queries muito parecidas
if to_replace:
    print(f"🔄 Substituindo {len(to_replace)} queries muito similares...")
    for index in sorted(to_replace, reverse=True):  # Substituir de trás para frente para evitar problemas de índice
        _, text, q_type = selected_queries[index]
        
        # Busca uma alternativa no mesmo tipo
        available_queries = [q for q in query_types[q_type] if q not in selected_queries]
        if available_queries:
            replacement = random.choice(available_queries)
            print(f"✅ Substituindo: {text} → {replacement[1]}")
            selected_queries[index] = replacement
        else:
            # Se não houver substituto, apenas remove
            print(f"⚠️ Nenhuma substituição disponível para: {text}. Removendo.")
            selected_queries.pop(index)


# Exibindo resultado
print(f"Total de queries válidas: {len(queries_annotated)}")
print(f"Total de queries selecionadas: {len(selected_queries)}")

# Exemplo de saída das queries selecionadas
for query_id, query_text,query_type in selected_queries[:10]:  # Exibir 10 exemplos
    print(f"Query ID: {query_id} | Texto: {query_text} | Tipo: {query_type}")


Dataset carregado com sucesso!


ERROR:root:Erro ao iterar sobre as queries: 'charmap' codec can't decode byte 0x9d in position 6067: character maps to <undefined>


🔄 Queries muito similares detectadas:
  [1] who determines disability for social security
  [2] how much is social security for disability
------------------------------------------------------------
🔄 Queries muito similares detectadas:
  [1] who was scheherazade
  [2] who is scheherazade
------------------------------------------------------------
🔄 Queries muito similares detectadas:
  [1] average cost of assisted living in washington state
  [2] average cost of assisted living in kansas
------------------------------------------------------------
🔄 Queries muito similares detectadas:
  [1] how much is social security for disability
  [2] social security's definition of disability
------------------------------------------------------------
🔄 Queries muito similares detectadas:
  [1] gastric bypass cost
  [2] gastric bypass surgery cost
------------------------------------------------------------
🔄 Substituindo 5 queries muito similares...
✅ Substituindo: social security's definitio

In [71]:
NUM_DOCS_PER_QUERY = 3
query_to_docs = {}  # {query_id: [doc_id, doc_text]}

# Mapear queries para documentos relevantes
for qrel in dataset.qrels_iter():
    if qrel.query_id in [q[0] for q in selected_queries]:  # Apenas queries selecionadas
        if qrel.query_id not in query_to_docs:
            query_to_docs[qrel.query_id] = []
        query_to_docs[qrel.query_id].append(qrel.doc_id)

# Coletar os textos dos documentos
doc_texts = {}  # {doc_id: doc_text}
for doc in dataset.docs_iter():
    if doc.doc_id in {doc_id for docs in query_to_docs.values() for doc_id in docs}:  # Apenas docs necessários
        doc_texts[doc.doc_id] = decode_utf8_safe(doc.text)

# Criar JSONL para treino do modelo
with open("train_data.jsonl", "w", encoding="utf-8") as f:
    for query_id, query_text, query_type in selected_queries:
        doc_list = query_to_docs.get(query_id, [])[:NUM_DOCS_PER_QUERY]  # Seleciona até 5 docs
        for doc_id in doc_list:
            f.write(json.dumps({
                "query_id": query_id,
                "query_text": query_text,
                "query_type": query_type,
                "doc_id": doc_id,
                "doc_text": doc_texts.get(doc_id, "")
            }) + "\n")

print(f"Dataset salvo com {len(selected_queries)} queries e {sum(len(docs) for docs in query_to_docs.values())} documentos.")

KeyboardInterrupt: 

Objetivo: criar uma rotina que gere um conjunto de dados válido de menor escala

Passo a passo:

- Pegar só o que é escrito no nosso alfabeto (utf-8)

- Ordenar por tipo de pergunta

- Selecionar quantidades  iguais de cada tipo

- Trocar perguntas similares usando TF-IDF

- Escolher quais documentos serão utilizados usando algum critério (A ser feito)

- Salvar em um formato mais leve para processar os dados (A ser feito)
